# Iterative Solvers

The purpose of this notebook is to compare commonly available iterative solvers from python. In this notebook we will try CG with ILU and AMG preconditioning.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as spla

import ibmos as ib

In [3]:
import scipy.linalg as la

# Case initialization
... from [Unsteady flow around cylinder (Re=200)](../1-Basic/14-CylinderRe200.ipynb)

In [4]:
s1 = ib.stretching(256, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
s2 = ib.stretching(128, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
x = np.r_[-s2[::-1], s1[1:]]

s = ib.stretching(192, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
y = np.r_[-s[::-1], s[1:]]

solver = ib.Solver(x, y, iRe=1/200, Co=0.015/0.033)
del x, y, s1, s2

solver.set_solids(ib.shapes.cylinder("cylinder", 0, 0, 0.5, solver.dxmin))

uBC, vBC = solver.zero_boundary_conditions()
for k in range(4):
    uBC[k][:] = 1

sBC = ((np.zeros(solver.solids[0].l), np.zeros(solver.solids[0].l)), )

u, v, p, *f = solver.reshape(*solver.unpack(solver.zero()))
c, r, ϵ = -10, 1.0, 0.01
u[:,:] = 1.0-ϵ*solver.fluid.u.y[:,np.newaxis]*np.exp(-((solver.fluid.u.x[np.newaxis,:]-c)**2 + solver.fluid.u.y[:,np.newaxis]**2)/r**2)
v[:,:] = ϵ*(solver.fluid.v.x[np.newaxis,:]-c)*np.exp(-((solver.fluid.v.x[np.newaxis,:]-c)**2 + solver.fluid.v.y[:,np.newaxis]**2)/r**2)
x0 = solver.pack(u, v, p, f)

In [5]:
n = 10

# Conjugate gradient with ILU preconditioning

In [6]:
solver.set_solver(ib.tools.solver_pcg_ilu)

## With fractional step method

In [7]:
solver.set_fractional_step(True)

In [8]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39397e+03  8.94209e+04  2.32796e+02  1.36951e-04  8.56479e-06  3.19661e-06 
CPU times: user 1min 40s, sys: 2.92 s, total: 1min 42s
Wall time: 12.4 s


In [9]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94825e+02  8.88276e+04  2.12360e+00  5.47619e-05  8.55968e-06  9.10884e-06 
       2  3.00000e-02  3.90542e+02  1.24832e+03  1.98460e+00  1.87335e-05  7.65063e-09  8.24982e-06 
       3  4.50000e-02  3.87182e+02  1.21734e+03  1.84205e+00  3.39496e-05  6.80074e-09  5.79607e-06 
       4  6.00000e-02  3.85509e+02  7.58487e+02  1.72137e+00  1.93097e-05  6.17337e-09  3.71450e-06 
       5  7.50000e-02  3.84971e+02  2.79749e+02  1.62180e+00  4.15206e-05  5.69216e-09  8.39675e-06 
       6  9.00000e-02  3.84841e+02  8.26313e+01  1.54095e+00  4.97579e-05  5.29113e-09  8.51065e-06 
       7  1.05000e-01  3.84594e+02  1.40943e+02  1.47505e+00  2.49146e-05  4.94732e-09  8.96527e-06 
       8  1.20000e-01  3.84155e+02  2.65012e+02  1.41984e+00 -1.10942e-05  4.70866e-09  9.23440e-06 
       9  1.35000e-01  3.83747e+02  2.79384e+02  1.37221e+00 -1.61612e-06  4.48183e-09  7.0

# Conjugate gradient with AMG preconditioning

In [10]:
solver.set_solver(ib.tools.solver_pcg_amg)

## With fractional step method

In [11]:
solver.set_fractional_step(True)

In [12]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -2.13774e-04  6.25158e-09  5.43127e-06 
CPU times: user 3min 7s, sys: 4.96 s, total: 3min 12s
Wall time: 7.63 s


In [13]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88306e+04  2.11840e+00  2.01780e-04  8.29921e-09  6.86713e-06 
       2  3.00000e-02  3.90525e+02  1.24727e+03  1.98458e+00 -3.29123e-05  4.90316e-10  7.14006e-06 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84209e+00  3.72261e-06  3.97435e-10  8.98137e-06 
       4  6.00000e-02  3.85515e+02  7.64251e+02  1.72136e+00  3.23033e-05  3.43289e-10  6.57466e-06 
       5  7.50000e-02  3.84973e+02  2.80605e+02  1.62183e+00 -2.36822e-05  3.04427e-10  5.41876e-06 
       6  9.00000e-02  3.84840e+02  8.37238e+01  1.54097e+00  3.95914e-06  2.76905e-10  8.82681e-06 
       7  1.05000e-01  3.84593e+02  1.40875e+02  1.47506e+00 -3.66370e-06  2.53553e-10  6.35729e-06 
       8  1.20000e-01  3.84154e+02  2.64532e+02  1.41991e+00  1.03543e-05  2.33640e-10  8.76132e-06 
       9  1.35000e-01  3.83768e+02  2.56008e+02  1.37222e+00  8.61080e-06  2.21293e-10  6.9